# ![](https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png) Project 3: Web APIs & NLP


--- 

*Group 3* | *Team Members: Constance, Wenzhe, Matthew, Joel*

### <b> Notebook 1: Data Scraping & Cleaning </b>

<b> (a) Overview </b>

Codes in this notebook
* Scrape 1,000 "hottest" threads & accompanying comments from (a) r/coffee and (b) r/tea subreddits
* Clean scraped data and save them as separate "coffee" and "tea" datasets

<b> (b) Structure of Notebook 1 </b>

* Part 1: Data Collection - Scrapping Subreddit Data
* Part 2. EDA : To Uncover Potential Data Issues Requiring Cleaning
* Part 3. Data Cleaning : Removing Dataframe Rows that are Not Useful/Errorneous
* Part 4. Merge Comments & Threads Dataframes
* Part 5. Save Cleaned Merged Dataframes as CSV Files 


---

### Import Libraries

In [1]:
# 1. Installation
#!pip install praw

# 2. Imports
import praw
import pandas as pd
import re

---

### Part 1: Data Collection - Scrapping Subreddit Data

Objective

* Scrape 1,000 "hottest" threads & accompanying comments from (a) r/coffee and (b) r/tea subreddits
* Based on a review of subreddits online, we expect multiple comments per thread - hence, the number of each subreddit's accompanying comments >1,000

Note on Codes Presented in this Notebook Section

* A long duration is required for the data to be scrapped in full 
* As such, the codes below illustrate scrapping of a subset of data, namely: 100 threads and accompanying comments from "r/coffee", to provide proof of their workability

This code section produces 4 files: 

|No.|Description|Filename|
|---|---|---|
|1|r/coffee: threads|raw_orig_coffee_threads.csv|
|2|r/coffee: accompanying comments|raw_orig_coffee_comments.csv|
|3|r/tea: threads|raw_orig_tea_threads.csv|
|4|r/tea: accompanying comments|raw_orig_tea_comments.csv|

(a) Scrape Data

* To replicate our scraping procedure:
    * toggle between "coffee" and "tea" in code line: `subreddit = reddit.subreddit("coffee")`
    * change limit from 100 to 1,000 in code line: `for submission in subreddit.hot(limit = 100)` 

In [5]:
# authenticate with Reddit API
reddit = praw.Reddit(
    client_id = "", # ID and Key has been removed from the notebook, run using the csv files below instead, and skip sections (a) and (b)
    client_secret = "",
    user_agent = "kopicoffee",
    ratelimit_seconds = 30)

# create 2 separate lists to store threads and comments after we scrape them from the subreddit
threads = []
comments = []

# subreddit to scrape
# r/tea = "tea"
# r/Coffee = "coffee"
subreddit = reddit.subreddit("coffee")

# iterate through hottest threads in subreddit
# scrape 100 threads (for illustrative purposes: to reset to 1,000) 
for submission in subreddit.hot(limit = 100):
    try:
        tr_author_nm = submission.author.name
    except:
        #do nothing
        tr_author_nm = ""
    if hasattr(submission, "post_hint"):
        hint = submission.post_hint
    else:
        hint = ""


    # store thread data
    thread = {
        "id": submission.id,
        "title": submission.title, 
        "score": submission.score,
        "num_comments": submission.num_comments,
        "post_hint": hint,
        "self_text": submission.selftext,
        "author_name": tr_author_nm,
        "url": submission.url # can use this to check for i.reddit / v.reddit / gallery
    }  
    threads.append(thread)
    

    # store all comment data under thread
    submission.comments.replace_more(limit=0)
    for comment in submission.comments.list():
        # required as there are some comments without author name
        author_nm = ""
        try:
            author_nm = comment.author.name
        except:
            #do nothing
            author_nm = ""
        # store thread data
        comment_data = {
            "thread_id": submission.id, 
            "comment_id": comment.id,
            "comment_text": comment.body,
            "comment_score" : comment.score,
            "author_name": author_nm
        }
        comments.append(comment_data)

(b) Convert Lists to Pandas Dataframes, and Export Dataframes as CSV files

* To replicate our conversion procedure, toggle between:
    * tea_threads, tea_comments
    * coffee_threads, coffee_comments

In [6]:
# convert the lists in pandas dataframes
coffee_threads_df = pd.DataFrame(threads)
coffee_comments_df = pd.DataFrame(comments)

# # save a copy of raw data for comments and threads
coffee_threads_df.to_csv("data/raw_orig_coffee_threads_test.csv", index=False)
coffee_comments_df.to_csv("data/raw_orig_coffee_comments_test.csv", index=False)

(c) Check Info of the Raw Dataframes

In [2]:
# Read in fully scrapped files
coffee_threads_df  = pd.read_csv("data/raw_orig_coffee_threads.csv")
coffee_comments_df = pd.read_csv("data/raw_orig_coffee_comments.csv")

tea_threads_df  = pd.read_csv("data/raw_orig_tea_threads.csv")
tea_comments_df = pd.read_csv("data/raw_orig_tea_comments.csv")

In [3]:
# Check info of the raw dataframe 
# > Illustratively: Coffee Threads
coffee_threads_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 674 entries, 0 to 673
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            674 non-null    object
 1   title         674 non-null    object
 2   score         674 non-null    int64 
 3   num_comments  674 non-null    int64 
 4   post_hint     41 non-null     object
 5   self_text     664 non-null    object
 6   author_name   672 non-null    object
 7   url           674 non-null    object
dtypes: int64(2), object(6)
memory usage: 42.3+ KB


In [4]:
# Check info of the raw dataframe 
# > Illustratively: Coffee Comments
coffee_comments_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32172 entries, 0 to 32171
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   thread_id      32172 non-null  object
 1   comment_id     32172 non-null  object
 2   comment_text   32172 non-null  object
 3   comment_score  32172 non-null  int64 
 4   author_name    31011 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.2+ MB



---

### Part 2. EDA : To Uncover Potential Data Issues Requiring Cleaning

Overview of Potential Data Issues Uncovered via EDA

(a) Threads: threads with media content <br>
(b) Threads: non-discussion threads <br>
(c) Comments: text that shows up as [deleted] and have no corresponding author name [removed]


<b>(a) Threads: threads with media content</b>

This code sub-section:
* Explores the nature of threads with media content
* Adds a new variable to identify threads with media content

EDA Threads : Columns and Contents

In [5]:
# 1. Examine threads_df to view the columns and contents
# > Illustratively: Coffee Threads
coffee_threads_df.head()

,id,title,score,num_comments,post_hint,self_text,author_name,url
0,19agk2c,[MOD] The Daily Question Thread,2,58,NaN,\n\nWelcome to the daily [/r/Coffee](https://...,menschmaschine5,https://www.reddit.com/r/Coffee/comments/19agk...
1,19ambi4,[MOD] What have you been brewing this week?/ C...,25,46,NaN,Hey everyone!\n\nWelcome back to the weekly /r...,menschmaschine5,https://www.reddit.com/r/Coffee/comments/19amb...
2,19a84l7,"Drying out Coffee overnight, worth it?",42,38,NaN,Every time my mother gets a new bag of coffee ...,a_n_d_r_e_w,https://www.reddit.com/r/Coffee/comments/19a84...
3,199tgjn,Convention Booth questions,25,7,NaN,I have an opportunity to have a booth at a loc...,ExistentialJew,https://www.reddit.com/r/Coffee/comments/199tg...
4,199yiv0,macchiato confusion,3,7,NaN,Hello! My apologies if this question is dumb....,Sudden-Ice-9613,https://www.reddit.com/r/Coffee/comments/199yi...


EDA Threads : "post_hint" Column

In [6]:
# 2. Examine "post_hint" column (Illustratively: Coffee Comments)
# > Illustratively: Coffee Threads
coffee_threads_df["post_hint"].unique()

array([nan, 'self', 'image', 'link', 'rich:video', 'hosted:video'],
      dtype=object)

Feature Construction : Create New Column to Identify Media Threads

* `post_hint` is Reddit's way of categorising media threads
* However, when we checked the csv files vs the actual post online, we found that it is not very accurate
* Hence we decided to create a new column, `url_is_media`, to identify media threads

In [7]:
# Create a function to filter out non text i.e. images 
# if it is non text, categorize as 0; if media, categorize as 1
# note: i.reddit and v.reddit indicate the media is hosted directly on reddit

def is_media_url(text):
    media_url_regex = r"https://(?:i\.redd\.it/|v\.redd\.it/|www\.reddit\.com/gallery/|imgur\.com/a/)[^\s]+"
    return 1 if re.match(media_url_regex, text) else 0

In [8]:
# Create a new column to indicate if there is a media posted in the thread, map the function to the url
coffee_threads_df["url_is_media"] = coffee_threads_df["url"].apply(lambda x: is_media_url(x))
tea_threads_df["url_is_media"] = tea_threads_df["url"].apply(lambda x: is_media_url(x))

In [9]:
# Check our updated dataframe for addition of "url_is_media" variable 
# > Illustratively: Coffee Threads
coffee_threads_df.head()

,id,title,score,num_comments,post_hint,self_text,author_name,url,url_is_media
0,19agk2c,[MOD] The Daily Question Thread,2,58,NaN,\n\nWelcome to the daily [/r/Coffee](https://...,menschmaschine5,https://www.reddit.com/r/Coffee/comments/19agk...,0
1,19ambi4,[MOD] What have you been brewing this week?/ C...,25,46,NaN,Hey everyone!\n\nWelcome back to the weekly /r...,menschmaschine5,https://www.reddit.com/r/Coffee/comments/19amb...,0
2,19a84l7,"Drying out Coffee overnight, worth it?",42,38,NaN,Every time my mother gets a new bag of coffee ...,a_n_d_r_e_w,https://www.reddit.com/r/Coffee/comments/19a84...,0
3,199tgjn,Convention Booth questions,25,7,NaN,I have an opportunity to have a booth at a loc...,ExistentialJew,https://www.reddit.com/r/Coffee/comments/199tg...,0
4,199yiv0,macchiato confusion,3,7,NaN,Hello! My apologies if this question is dumb....,Sudden-Ice-9613,https://www.reddit.com/r/Coffee/comments/199yi...,0


<b>(b) Threads: non-discussion threads</b>

Context: 
* During the process of reviewing the subreddits' threads, we uncovered marketing threads created by "automoderator bots" and "moderators".

This code sub-section adds new variables to identify:
* Marketing threads by AutoModerator bots
* Marketing threads by moderators in the subreddit


In [10]:
# Coffee

# 1. Add identifier: threads by thread_id posted by automoderator bots
coffee_marketing_thread_ids_automod = list(coffee_threads_df.loc[(coffee_threads_df["author_name"] == "AutoModerator") & (coffee_threads_df["title"].str.startswith("Marketing"))]["id"])
    
# 2. Add identifier: threads by thread_id posted by moderators
coffee_marketing_thread_ids_mod = list(coffee_threads_df.loc[coffee_threads_df["title"].str.contains("The Official Deal Thread")]["id"])

# 3. Consolidate IDs
coffee_marketing_thread_ids = coffee_marketing_thread_ids_automod.copy()

for id in coffee_marketing_thread_ids_mod:
    if id not in coffee_marketing_thread_ids:
        coffee_marketing_thread_ids.append(id)
        

In [11]:
# Tea

# 1. Add identifier: threads by thread_id posted by automoderator bots
tea_marketing_thread_ids_automod = list(tea_threads_df.loc[(tea_threads_df["author_name"] == "AutoModerator") & (tea_threads_df["title"].str.startswith("Marketing"))]["id"])

# 2. Add identifier: threads by thread_id posted by moderators
tea_marketing_thread_ids_mod = list(tea_threads_df.loc[tea_threads_df["title"].str.contains("The Official Deal Thread")]["id"])

# 3. Consolidate IDs
tea_marketing_thread_ids = tea_marketing_thread_ids_automod.copy()

for id in tea_marketing_thread_ids_mod:
    if id not in tea_marketing_thread_ids:
        tea_marketing_thread_ids.append(id)
        

<b>(c) Comments: text that shows up as [deleted] and have no corresponding author name [removed] </b>

EDA: Examine No. of Comments & Corresponding Authors

In [12]:
# Check the comment dataframe to see columns and row contents 
# > Illustratively: Coffee Comments
coffee_comments_df

,thread_id,comment_id,comment_text,comment_score,author_name
0,19agk2c,kikp8lv,Hi! I’m a morning coffee drinker but I never m...,2,testingpage2025
1,19agk2c,kikxhmz,Any Scooters employees around? I need to know ...,2,Ok_Bet_2634
2,19agk2c,kinuq9z,Do any of you ever get “too lazy” to make coff...,2,angelsandairwaves93
3,19agk2c,kil6vl0,I can't find a pumpkin spice latte recipe that...,1,automirage04
4,19agk2c,kilaorz,Hey all! I live in Costa Rica and regularly bu...,1,chuvakinfinity
...,...,...,...,...,...
32167,13f5sqf,jjtdirv,"There's tons of info out there, but you need t...",3,AnimorphsGeek
32168,13f5sqf,jju4t0r,Books written by Michael Sivets tend to be a p...,3,LATechSpartan
32169,13f5sqf,jk1k4n8,https://instagram.com/espressofun?igshid=ZWIzM...,1,PoJenkins
32170,13f5sqf,jjtnoon,"""The Physics of Filter Coffee"" is pretty good ...",15,HR_Paul


In [13]:
# Check no. of comments and corresponding no. of authors
# > Illustratively: Coffee Comments

print("Coffee: Comments")
print("No. of comments:", coffee_comments_df.shape[0])
print("No. of authors:",coffee_comments_df["author_name"].nunique())

Coffee: Comments
No. of comments: 32172
No. of authors: 9367


EDA: Examine No. of Comments per Author

In [14]:
# group by author_name to see who contributed the most in the comments across all threads
# authors with highest count displayed first
# > Illustratively: Coffee Comments

coffee_comments_df.groupby("author_name").size().sort_values(ascending=False)

author_name
LEJ5512                1022
Anomander               716
paulo-urbonas           566
VibrantCoffee           440
p739397                 403
                       ... 
SeveralSecurity8058       1
Serpents_disobeyed        1
Serious_Dot_4532          1
SeriousJak                1
zzzz_ok                   1
Length: 9367, dtype: int64

Comments:

From the results above, and examining "coffee comments" for illustrative purposes: 

* We find that there are only ~9,400 authors despite having ~32,000 comments
* While there is a possibility of an author having multiple comments, the difference is quite big
* There could be a significant portion of comments with no author name

Hence we checked the csv and realised that there are deleted comments represented by "[deleted]" with no author names, which presents a comment missingness concern
* We double checked that with the code below

In [15]:
# check author names of deleted comments
# > Illustratively: Coffee Comments

coffee_comments_df[coffee_comments_df["comment_text"] == "[deleted]"]["author_name"].unique()

array([nan], dtype=object)


---

### Part 3. Data Cleaning : Removing Dataframe Rows that are Not Useful/Errorneous

* Based on the above potential issues, we will remove them from the existing dataframe but save the new dataframe into a new variable

<b> (a) Create new "clean" dataframe variables </b>

In [16]:
# create new "clean" dataframe variables
df_coffee_threads_clean   = coffee_threads_df
df_coffee_comments_clean = coffee_comments_df 

df_tea_threads_clean   = tea_threads_df  
df_tea_comments_clean = tea_comments_df 

# print: initial no. of df rows
print("Coffee - Threads: n(rows) =", df_coffee_threads_clean.shape[0])
print("Coffee - Comments: n(rows) =", df_coffee_comments_clean.shape[0])
print("")
print("Tea - Threads: n(rows) =", df_tea_threads_clean.shape[0])
print("Tea - Comments: n(rows) =", df_tea_comments_clean.shape[0])

Coffee - Threads: n(rows) = 674
Coffee - Comments: n(rows) = 32172

Tea - Threads: n(rows) = 962
Tea - Comments: n(rows) = 12645


<b> (b) Removal of non discussion threads and their associated comments (based on part 2b) </b>

As these threads are marketing related, it is not useful in our sentiment analysis or modelling hence we will remove them.

In [17]:
# Coffee

# 1. remove from threads dataframe
mask_marketing_threads = df_coffee_threads_clean["id"].isin(coffee_marketing_thread_ids)
df_coffee_threads_clean = df_coffee_threads_clean[~mask_marketing_threads]

# 2. remove from comments dataframe
mask_marketing_threads = df_coffee_comments_clean["thread_id"].isin(coffee_marketing_thread_ids)
df_coffee_comments_clean = df_coffee_comments_clean[~mask_marketing_threads]

# 3. print no. of df rows
print("Coffee - Threads: n(rows) =", df_coffee_threads_clean.shape[0])
print("Coffee - Comments: n(rows) =", df_coffee_comments_clean.shape[0])

Coffee - Threads: n(rows) = 649
Coffee - Comments: n(rows) = 31842


In [18]:
# Tea

# 1. remove from threads dataframe
mask_marketing_threads = df_tea_threads_clean["id"].isin(tea_marketing_thread_ids)
df_tea_threads_clean = df_tea_threads_clean[~mask_marketing_threads]

# 2. remove from comments dataframe
mask_marketing_threads = df_tea_comments_clean["thread_id"].isin(tea_marketing_thread_ids)
df_tea_comments_clean = df_tea_comments_clean[~mask_marketing_threads]

# 3. print no. of df rows
print("Tea - Threads: n(rows) =", df_tea_threads_clean.shape[0])
print("Tea - Comments: n(rows) =", df_tea_comments_clean.shape[0])

Tea - Threads: n(rows) = 959
Tea - Comments: n(rows) = 12605


<b> (c) Removal of rows with comment text as [deleted] (based on part 2c) </b>

As there is no comment content, it is not useful in our modelling hence we will remove them

In [19]:
# create a mask variable to contain rows with "deleted" or "removed" based on part (2c)
deleted_texts = ["[deleted]", "[removed]"]

# 1. remove them from dataframe = coffee
df_coffee_comments_clean = df_coffee_comments_clean[~df_coffee_comments_clean["comment_text"].isin(deleted_texts)]

# 2. remove them from dataframe = tea
df_tea_comments_clean = df_tea_comments_clean[~df_tea_comments_clean["comment_text"].isin(deleted_texts)]

# 3. print no. of df rows
print("Coffee - Comments: n(rows) =", df_coffee_comments_clean.shape[0])
print("Tea - Comments: n(rows) =", df_tea_comments_clean.shape[0])

Coffee - Comments: n(rows) = 31235
Tea - Comments: n(rows) = 12520


<b> (d) Remove Reddit's own markdown formatting for links and keep only the text </b>

We only want to keep the text from the links as the text could possibly give an insight into what the contents are in the related link

[] Test code

In [20]:
# testing out the regex expression to see if we filtered out the rows correctly
# > Illustratively: Coffee Comments

link_regex = r'\[(.+?)\]\((.+?)\)'
df_coffee_comments_clean[df_coffee_comments_clean['comment_text'].str.match(link_regex)].head()

,thread_id,comment_id,comment_text,comment_score,author_name
59,19ambi4,kimb1hx,[Touchy Ethiopia habtamu fikadu](https://touch...,5,vicerowv86
63,19ambi4,kim52y1,[Edwin Zambrano - 170hr Anaerobic Natural](htt...,2,anaerobic_natural
71,19ambi4,kimjuew,[Night fly roast works](https://nightflyerroas...,1,hippihoopihoo
80,19ambi4,kine3y5,[https://regaliacoffee.com/products/rainbow-ho...,1,ad6499
154,199sf8j,kigvg2o,[Going for versatility here](https://i.imgur.c...,2,DrSpaceman575


[] Define filtering function

In [21]:
# define a function to filter out rows with markdown formatting on links

def keep_text_from_link(text):
    link_regex = r'\[(.+?)\]\((.+?)\)'
    
    def replace_link(match):
        return match.group(1)

    return re.sub(link_regex, replace_link, text)

[] Apply filtering function

In [22]:
# apply to the coffee and tea dataframes
df_coffee_comments_clean["comment_text"] = df_coffee_comments_clean["comment_text"].apply(lambda x: keep_text_from_link(x))
df_tea_comments_clean["comment_text"]    = df_tea_comments_clean["comment_text"].apply(lambda x: keep_text_from_link(x))

[] Check if filtering is successful
* check again to see if any links have been unprocessed by our function
* should return with no results if function successfully applied to all relevant rows

In [23]:
df_coffee_comments_clean[df_coffee_comments_clean["comment_text"].str.match(link_regex)].head()

,thread_id,comment_id,comment_text,comment_score,author_name


In [24]:
df_tea_comments_clean[df_tea_comments_clean["comment_text"].str.match(link_regex)].head()

,thread_id,comment_id,comment_text,comment_score,author_name


<b> (e) Removal of url-only comments </b>

We do not want them because they do not provide any form of sentiment analysis nor use in modelling

[] Test code

In [25]:
# testing out the regex expression to see if we filtered out the rows correctly
# > Illustratively: Coffee Comments

url_regex = r'https?://\S+|www\.\S+'
df_coffee_comments_clean[df_coffee_comments_clean["comment_text"].str.match(url_regex)]

,thread_id,comment_id,comment_text,comment_score,author_name
80,19ambi4,kine3y5,https://regaliacoffee.com/products/rainbow-hop...,1,ad6499
702,19764n9,ki2mk46,https://us.amazon.com/Coffee-Organizer-Silicon...,2,Vietname
989,1950ew2,khkhzw9,https://tendropscoffee.com/\n\nI just picked u...,1,Burninmules
1457,192sur5,khotlky,https://www.oxo.com/barista-brain-9-cup-coffee...,2,wasdtomove
1516,1935pxp,kha2kh6,https://www.reddit.com/r/FlairEspresso/s/HLKhG...,1,p739397
...,...,...,...,...,...
30936,13m2h59,jksx6a7,https://iccontrols.com/faq-items/what-is-the-d...,1,EXTRA370H55V
31804,13hjhmw,jk5wj0v,https://www.baristahustle.com/blog/diy-water-r...,5,Josefest
31847,13h86yx,jk6cr7l,https://youtu.be/au92VvBYLrA,1,paulo-urbonas
32121,13fgl41,jjvsnp5,https://i.imgur.com/ekiK6j2.jpg\nWhat can thes...,1,Taskolo


[] Define filtering function

In [26]:
# define a function to filter out rows with urls and return with a blank instead
def remove_url(text):
    url_regex = r'https?://\S+|www\.\S+'
    return re.sub(url_regex, '', text)

[] Apply filtering function

In [27]:
# apply to the coffee and tea dataframes
df_coffee_comments_clean["comment_text"] = df_coffee_comments_clean["comment_text"].apply(lambda x: remove_url(x))
df_tea_comments_clean["comment_text"] = df_tea_comments_clean["comment_text"].apply(lambda x: remove_url(x))

[] Check if filtering is successful
* check again to see if any rows have been unprocessed by our function
* should return with no results if function successfully applied to all relevant rows

In [28]:
df_coffee_comments_clean[df_coffee_comments_clean['comment_text'].str.match(url_regex)]

,thread_id,comment_id,comment_text,comment_score,author_name


In [29]:
df_tea_comments_clean[df_tea_comments_clean['comment_text'].str.match(url_regex)]

,thread_id,comment_id,comment_text,comment_score,author_name


<b> (f) Removal of comments posted by AutoModerator</b>

We do not want them because they are not posted by the users of the subreddit, and hence not useful in sentiment analysis or modelling

[] Apply filtering

In [30]:
# only save comment rows that are not posted by automoderator
df_coffee_comments_clean = df_coffee_comments_clean[df_coffee_comments_clean["author_name"] != "AutoModerator"]
df_tea_comments_clean = df_tea_comments_clean[df_tea_comments_clean["author_name"] != "AutoModerator"]

[] Check if filtering is successful
* sanity check on updated dataframe to see if any comments by automoderator left

In [31]:
df_coffee_comments_clean[df_coffee_comments_clean["author_name"] == "AutoModerator"]

,thread_id,comment_id,comment_text,comment_score,author_name


In [32]:
df_tea_comments_clean[df_tea_comments_clean["author_name"] == "AutoModerator"]

,thread_id,comment_id,comment_text,comment_score,author_name


<b> (g) Removal of comments posted by Bots</b>

* A review of the data shows that there are comments posted by bots
* We do not want them because they are not posted by the users of the subreddit, and hence not useful in sentiment analysis or modelling

In [33]:
# remove comments from bots

bot_list = ["alphabet_order_bot", 
            "Booty_Warrior_bot", 
            "VettedBot", 
            "TheGratitudeBot", 
            "exclaim_bot",
            "FakespotAnalysisBot",
            "RemindMeBot",
            "sneakpeekbot",
            "subredditsummarybot",
            "of_patrol_bot",
            "SexPanther_Bot",
            "lerobinbot"]

for bot in bot_list: 
    df_coffee_comments_clean = df_coffee_comments_clean.drop(df_coffee_comments_clean.loc[(df_coffee_comments_clean["author_name"] == bot),:].index , axis=0)
    df_tea_comments_clean = df_tea_comments_clean.drop(df_tea_comments_clean.loc[(df_tea_comments_clean["author_name"] == bot),:].index , axis=0)

In [34]:
# print no. of df rows
print("Coffee - Comments: n(rows) =", df_coffee_comments_clean.shape[0])
print("Tea - Comments: n(rows) =", df_tea_comments_clean.shape[0])

Coffee - Comments: n(rows) = 31227
Tea - Comments: n(rows) = 12201


<b> (h) Removal of rows with "blank" comments </b>

This sub-section: 
* first checks for and examines the nature of rows with empty/blank comments
* removes any rows with empty/blank comments

In [35]:
print("Coffee comments - no. of empty comments:", df_coffee_comments_clean["comment_text"].isnull().sum())
print("Tea comments - no. of empty comments:", df_tea_comments_clean["comment_text"].isnull().sum())

Coffee comments - no. of empty comments: 0
Tea comments - no. of empty comments: 0


<b>(i) Review : Dataframe Row Counts</b>

In [36]:
# print: no. of df rows
print("Coffee - Threads: n(rows) =", df_coffee_threads_clean.shape[0])
print("Coffee - Comments: n(rows) =", df_coffee_comments_clean.shape[0])
print("")
print("Tea - Threads: n(rows) =", df_tea_threads_clean.shape[0])
print("Tea - Comments: n(rows) =", df_tea_comments_clean.shape[0])

Coffee - Threads: n(rows) = 649
Coffee - Comments: n(rows) = 31227

Tea - Threads: n(rows) = 959
Tea - Comments: n(rows) = 12201



---

### Part 4. Merge Comments & Threads Dataframes

Overview of Dataframes:
* `df_coffee_comments_clean`, `df_tea_comments_clean` : dataframes that contain only rows with cleaned/useful comment text 
* `df_coffee_threads_clean`, `df_tea_threads_clean` : updated dataframes but may contain threads that we deemed not useful and have dropped off

Overview of Merging Process:
* Objective: produce coffee, tea "comments" datasets with accompanying thread info
* We will do a _left merge_ "threads_clean" [Right] onto "comments_clean" [Left] based on "thread_id", so that rows that are not in our cleaned comment dataframe will be excluded 
* This also helps us to map thread information to each comment so that we can further categorize comments based on which thread they belong to.

[] Perform merging

In [37]:
# perform merging
df_coffee_comments_clean_merged = pd.merge(df_coffee_comments_clean, df_coffee_threads_clean, left_on='thread_id', right_on='id', how='left', suffixes=('', '_thread'))
df_tea_comments_clean_merged = pd.merge(df_tea_comments_clean, df_tea_threads_clean, left_on='thread_id', right_on='id', how='left', suffixes=('', '_thread'))

[] Check: no. of comments & associated threads in each dataframe

In [38]:
# Obtain thread/comments stats

df_dict = {"Dataset: coffee":df_coffee_comments_clean_merged,
           "Dataset: tea":df_tea_comments_clean_merged}

for desc,df in df_dict.items():

    print(desc)
    
    # 1. No. of comments
    print("No. of comments:", round(df.shape[0],2))

    # 2. No. of threads
    print("No. of threads:", df["thread_id"].nunique())

    # 3. Ave. no. of comments per thread
    print("Average no. of comments per thread:", round(df.groupby(by="thread_id")["comment_id"].count().mean(),2))
    print("")

Dataset: coffee
No. of comments: 31227
No. of threads: 633
Average no. of comments per thread: 49.33

Dataset: tea
No. of comments: 12201
No. of threads: 878
Average no. of comments per thread: 13.9



[] Check: rows and columns, to assess if merge was performed appropriately

In [39]:
# check rows and columns if merge is done appropriately
# > Illustratively: Coffee Comments
df_coffee_comments_clean_merged.head()

,thread_id,comment_id,comment_text,comment_score,author_name,id,title,score,num_comments,post_hint,self_text,author_name_thread,url,url_is_media
0,19agk2c,kikp8lv,Hi! I’m a morning coffee drinker but I never m...,2,testingpage2025,19agk2c,[MOD] The Daily Question Thread,2,58,NaN,\n\nWelcome to the daily [/r/Coffee](https://...,menschmaschine5,https://www.reddit.com/r/Coffee/comments/19agk...,0
1,19agk2c,kikxhmz,Any Scooters employees around? I need to know ...,2,Ok_Bet_2634,19agk2c,[MOD] The Daily Question Thread,2,58,NaN,\n\nWelcome to the daily [/r/Coffee](https://...,menschmaschine5,https://www.reddit.com/r/Coffee/comments/19agk...,0
2,19agk2c,kinuq9z,Do any of you ever get “too lazy” to make coff...,2,angelsandairwaves93,19agk2c,[MOD] The Daily Question Thread,2,58,NaN,\n\nWelcome to the daily [/r/Coffee](https://...,menschmaschine5,https://www.reddit.com/r/Coffee/comments/19agk...,0
3,19agk2c,kil6vl0,I can't find a pumpkin spice latte recipe that...,1,automirage04,19agk2c,[MOD] The Daily Question Thread,2,58,NaN,\n\nWelcome to the daily [/r/Coffee](https://...,menschmaschine5,https://www.reddit.com/r/Coffee/comments/19agk...,0
4,19agk2c,kilaorz,Hey all! I live in Costa Rica and regularly bu...,1,chuvakinfinity,19agk2c,[MOD] The Daily Question Thread,2,58,NaN,\n\nWelcome to the daily [/r/Coffee](https://...,menschmaschine5,https://www.reddit.com/r/Coffee/comments/19agk...,0



---

### Part 5. Save Cleaned Merged Dataframes as CSV Files 

In [40]:
df_coffee_comments_clean_merged.to_csv("data/coffee_comments_clean_merged.csv", index=False)
df_tea_comments_clean_merged.to_csv("data/tea_comments_clean_merged.csv", index=False)